In [ ]:
!pip install opencv-python
!pip install Pillow
!pip install matplotlib

In [ ]:
import csv
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm # notebook用
import copy
import os
import math
import time
import numpy as np
import cv2
import io
from urllib.request import urlopen

In [16]:
######################
#URLデータ読み込み
######################

# URLのCSVデータ（インプット）
url_data = "output/url_sample.csv"

url_list = [] #url情報リストの格納用リスト
url_id = 0 #urlデータのの通し番号id

# URLのCSVデータ読み込み
with open(url_data) as f:
    for url in csv.reader(f):
        if url == []:
            continue
        else:
              #時刻が'17:53:09'のように秒以下が無い(.000000になっている)時は.000000を文字列に追加。
            if len(url[0]) == 19: 
                url[0] = url[0] + '.000000'        
            gaze_time = dt.datetime.strptime(url[0], '%Y-%m-%d %H:%M:%S.%f') #文字列を時刻に変換
            row = [url_id, gaze_time, (url[1], url[2]), float(url[5]), url[7]] #[url_id、時刻、(緯度、軽度)、方向、panoid]
            url_list.append(row)
            url_id += 1

In [ ]:
######################
#視線データ読み込み
######################

#視線CSVデータ（インプット）
gaze_data = "output/gaze.csv"

gp_dict={} #注視点格納用dict
gaze_id = 0 #視線データの通し番号id

with open(gaze_data) as f: #生データ
    for gaze in csv.reader(f):
        if gaze != []:
            #[時刻, 右目x, 右目y, 左目x, 左目y]
            row = [df[0],df[1],df[2],df[3],df[4]
            #データの欠損状況で場合分け
            if row[1] != 'nan' and row[3] != 'nan':
                x = (float(row[1])+float(row[3]))/2
                y = (float(row[2])+float(row[4]))/2
            elif row[1] != 'nan' and row[3] == 'nan':
                x = float(row[1])
                y = float(row[2])
            elif row[1] == 'nan' and row[3] != 'nan':
                x = float(row[3])
                y = float(row[4])
            else: #全欠損の場合は−１とする
                continue
            #時刻が'17:53:09'のように秒以下が無い(.000000になっている)時は.000000を文字列に追加。
            if len(row[0]) == 19:
                row[0] = row[0] + '.000000'
            gaze_time = dt.datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S.%f')
                   
            row = [gaze_time, (x,y), gaze_id]
            gp_dict[gaze_id] = row
            gaze_id += 1

In [ ]:
######################
#URLデータの視線データを紐付け
######################

#時刻をもとにURLデータに視線データを紐付ける。 
#さらに緯度経度から、GSV上の同じ地点におけるURLデータと視線データをまとめる。

url_list_point = [] #一地点におけるURLデータのリスト
gaze_list_point = [] #一地点における視線データのリスト

URL_lists = [] #各地点におけるURLデータのリストを格納するリスト
Gaze_lists = [] #各地点における視線データのリストを格納するリスト

url_id = None #URLデータの通し番号id
gaze_id = 0 #視線データの通し番号id

#ループ用の変数
pointid = 0 #地点ID
pre_url = url_list[0]
coordinate = url_list[0][2]
head = url_list[0][3]
panoid = url_list[0][4]

for url in url_list:
    gp_list_tmp = [] #
    # 時刻がURLデータの時刻を超えるまで、視線データをリストに追加
    while gp_dict[gaze_id][0] =< url[1]:
        gp_dict[gaze_id].append([pointid,url[0],url[2],url[4],url[3]])
        gp_list_tmp.append(gp_dict[gaze_id])
        gaze_id += 1
    #URLに紐づく視線データを、地点におけるリストに追加。
    gaze_list_point.append(copy.deepcopy(gp_list_tmp))
    if url[2] != coordinate: #緯度軽度が変わった場合
        #前の地点におけるリストを格納
        Gaze_list.append(copy.deepcopy(gaze_list_point))
        URL_list.append(copy.deepcopy(url_list_point))
        coordinate = url[2]
        panoid = url[4]
        gaze_list_point = [] #初期化
        url_list_point = [] #初期化
        pointid += 1
    url_id = url[0]
    head = url[3]
    url.append(pointid)
    url_list_point.append(copy.deepcopy(url))

In [ ]:
######################
#GoogleStreetViewAPI画像取得用関数
######################

api_key = "{ここにAPI keyを入力}"
size=[600, 265] #画面サイズ(1920, 848)の比率
fov=120 #水平視野角
pitch = 0 #カメラ角度

#指定したカメラ方向のgsv画像をAPIで取得する関数
def getAPIImg(pointid, latitude, longitude, heading, j):
    # streetviewのURLを作成して画像を取得
    g_url="https://maps.googleapis.com/maps/api/"
    g_url_sub=[]
    g_url_sub.append("streetview?location=" + str(latitude) + "," + str(longitude)) # 場所の指定
    g_url_sub.append("&size=" + str(size[0]) + "x" + str(size[1])) # 画像サイズ
    g_url_sub.append("&heading=" + str(heading)) # カメラの方位
    g_url_sub.append("&pitch=" + str(pitch)) # カメラ角度
    g_url_sub.append("&fov=" + str(fov)) # 水平視野角
    g_url_sub.append("&key=" + api_key) #api key
    for t in g_url_sub:
        g_url = g_url + t
    file =io.BytesIO(urlopen(g_url).read())
    img__ =Image.open(file)
    img_ = np.array(img__, dtype=np.uint8)
    img = cv2.cvtColor(img_, cv2.COLOR_RGB2BGR)
    return img

# 一地点の複数API画像をリストにして返す関数
def getAPIImgs(pointid, urls):
    images = []
    for url in urls:
        img = getAPIImg(pointid, url[2][0], url[2][1], url[3], url[0])
        images.append(img)
    return images

In [ ]:
######################
#パノラマ画像生成用関数
######################

size=[600, 265] #画面サイズ(1920, 848)の比率

#カメラパラメータ
focus = 200
scale = focus

#平面画像→パノラマ画像に変換するためのカメラ行列
K = np.array([[focus, 0, size[0]/2], [0, focus, size[1]/2], [0, 0, 1]], dtype=np.float32)

Rs_list = [] #平面画像→パノラマ画像に変換するための回転行列のリスト
origins = [] #パノラマ画像の原点のリスト
syuseikakus = [] #修正角のリスト
centers = [] #パノラマ画像の中心のリスト

#  カメラ方向が0度を跨ぐ場合の修正をするための関数
# 北向き=0度方向を向いている地点ではパノラマ画像が0度を境に分割されてしまうため、角度を調節して修正
def KakudoSyusei(urls):
    syuseikaku = 0 #修正回転角
    syusei = False
    while syusei is False:
        ok = 0 #0のままならOK
        if syuseikaku == 360:
            syusei = True
            break
        for i, url in enumerate(urls):
            if url[3]+syuseikaku <= 60 or 300 <= url[3]+syuseikaku and url[3]+syuseikaku <= 420:
                ok += 1
        if ok == 0:
            syusei = True
            break
        else:
            syuseikaku += 30
    url[3] += syuseikaku
    return syuseikaku

#パノラマ画像の中心を求める関数
def getCenter(urls):
    minx, maxx = 999999, -999999
    for i, url in enumerate(urls):
        if url[3] < minx: minx = url[3]
        if url[3] > maxx: maxx = url[3]
    center_x = (minx + maxx) / 2
    right_x = center_x - 180
    return center_x, right_x

#出力画像の原点の座標を求める関数
def getOrigin(corners):
    x_min = 9999999
    y_min = 9999999
    x_max = -9999999
    y_max = -9999999
    for c in corners:
        if c[0] < x_min: x_min = c[0]
        if c[1] < y_min: y_min = c[1]
        if c[0]+393 > x_max: x_max = c[0]+393
        if c[1]+234 > y_max: y_max = c[1]+234
    origin = [x_min, y_min, x_max, y_max]
    origins.append(getOrigin(corners))
    return origin

#平面画像→パノラマ画像に変換するための回転行列を求める関数
def getR(url):
    sin = math.sin(math.radians((url[3]-right_x)))
    cos = math.cos(math.radians((url[3]-right_x)))
    R = np.array([[-cos, 0, -sin], [0, 1, 0], [sin, 0, -cos]], dtype=np.float32)
    return R

#パノラマ画像の中心と右端の位置を求めリストに格納する関数
def culcPosition(urls):
    #北向き=0度方向を向いている地点ではパノラマ画像が0度を境に分割されてしまうため、角度を調節して修正する。
    syuseikaku = KakudoSyusei(urls)
    syuseikakus.append(syuseikaku)
    #この地点におけるパノラマ画像の中心と右端を求める。
    center_x, right_x = getCenter(urls)
    centers.append(center_x*math.pi/180)

# 各URLに対応する平面画像→パノラマ画像に変換するための回転行列を求めリストに格納する関数
def getRList(pointid, urls):
    for url in urls:
        R_list.append(getR(url))
    Rs_list.append(R_list)
    return R_list

#一地点におけるパノラマ画像を生成する関数
def genPanoImg(images, R_list):

    #画像のマッピング----------------------------------------
    corners = [] #マッピング後の左上の点の座標のリスト
    masks_warped = [] #画像のWarp()実行結果のリスト
    images_warped = [] #マスクのWarp()実行結果のリスト
    sizes = [] #Warp画像サイズのリスト
    masks = [] #初期化したマスク格納用リスト
    warper = cv2.PyRotationWarper('spherical', scale)
    #マスクの初期化
    for i in range(0, len(images)):
        um = cv2.UMat(255 * np.ones((images[i].shape[0], images[i].shape[1]), np.uint8))
        masks.append(um)
    #全画像とそのマスクに対して warper.warp() を実行して、結果を images_warped と masks_warped それぞれに格納
    for idx in range(0, len(images)):
        corner, image_wp = warper.warp(images[idx], K, R_list[idx], cv2.INTER_LINEAR, cv2.BORDER_REFLECT)
        corners.append(corner)
        sizes.append((image_wp.shape[1], image_wp.shape[0]))
        images_warped.append(image_wp)
        p, mask_wp = warper.warp(masks[idx], K, R_list[idx], cv2.INTER_NEAREST, cv2.BORDER_CONSTANT)
        masks_warped.append(mask_wp.get())
        
    #縫い目の算出----------------------------------------
    images_warped_f = []
    for img in images_warped:
        imgf = img.astype(np.float32)
        images_warped_f.append(imgf)
    seam_finder = cv2.detail_GraphCutSeamFinder('COST_COLOR')
    seam_finder.find(images_warped_f, corners, masks_warped)

    #ブレンド処理--------------------------------------------
    blend_strength = 5
    blender = cv2.detail.Blender_createDefault(cv2.detail.Blender_NO)
    dst_sz = cv2.detail.resultRoi(corners=corners, sizes=sizes)
    blend_width = np.sqrt(dst_sz[2] * dst_sz[3]) * blend_strength / 100
    if blend_width < 1:
        blender = cv2.detail.Blender_createDefault(cv2.detail.Blender_NO)
    blender.prepare(dst_sz)
    for idx in range(0, len(images)):
        images_warped_s = images_warped[idx].astype(np.int16)
        blender.feed(cv2.UMat(images_warped_s), masks_warped[idx], corners[idx])

    # #最終結果の出力--------------------------------------------
    result, result_mask = blender.blend(result, result_mask) #ブレンドする。
    pano_img = cv2.normalize(src=result, dst=None, alpha=255., norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U) #正規化
    
    return pano_img

In [ ]:
######################
#カーネル密度生成用関数
######################

import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import matplotlib.cm as cm
from matplotlib.colors import Normalize

#注視点のパノラマ画像上の極座標を求める関数
def getPolarCoords(gaze_point):
    #gaze_point = [gaze_time, [x,y,pupil], id, 地点ID, urlID, 緯度経度, panoID, head]
    x = gaze_point[1][0] * size[0]
    y = (gaze_point[1][1] - 172/1080) * size[1] #画面上部タブの172ピクセル分調整する
    X = r_kinv[0, 0] * x + r_kinv[0, 1] * y + r_kinv[0, 2]
    Y = r_kinv[1, 0] * x + r_kinv[1, 1] * y + r_kinv[1, 2]
    Z = r_kinv[2, 0] * x + r_kinv[2, 1] * y + r_kinv[2, 2]
    u = math.atan2(X, Y) #極座標u
    w = Y / math.sqrt(X**2 + Y**2 + Z**2)
    v = (math.pi - math.acos(w)) #極座標v
    return [u, v]

#一地点における全注視点のパノラマ画像上の極座標をリスト化して返す関数。
def getGazeListPano(pointid, gazes_list):
    gaze_list_pano = []
    for i, gazes in enumerate(gazes_list):
        kinv = np.linalg.inv(K) #カメラ行列の逆行列
        r_kinv = np.dot(Rs_list[pointid][i], kinv) #回転行列の逆行列
        if gazes is not None: #視点が記録されている場合
            for gaze_point in gazes:
                polar_coords = getPolarCoords(gaze_point) #注視点のパノラマ画像上の極座標
                gaze_list_pano.append(polar_coords)
    return gaze_list_pano
                
#カーネル密度のヒートマップを生成する関数
def gaussian(fix_arr, width, height, sigma=50):
    # fix_arr   : (x, y), (x, y),,,, ,,,(x, y), (x, y)
    # sigma  : gaussian Sd #正規分布の標準偏差
    x, y = np.arange(0, width, 1, float), np.arange(0, height, 1, float)
    x, y = np.meshgrid(x,y)
    heatmap = np.zeros((height, width), np.float32)
    for j in range(len(fix_arr)):
        heatmap += np.exp(-4*np.log(2) * ((x-fix_arr[j][0])**2 + (y-fix_arr[j][1])**2) / sigma**2)
    return heatmap

#正規化する関数
def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result

#カーネル密度を生成する関数
def Karnel(img, gaze_list_pano, pointid):
    img_x = img.shape[1]
    img_y = img.shape[0]
    gaze_list = np.array(gaze_list_pano)#注視点リストをnumpy配列に変換
    value = gaze_list.T #転置してgaussian_kdeに突っ込める形にする
    kernel = gaussian_kde(value, bw_method=0.7) #bw_method=バンド幅
    xx,yy = np.mgrid[0:img_x:1,0:img_y:1]
    meshdata = np.vstack([xx.ravel(),yy.ravel()])
    z = kernel.evaluate(meshdata)
    data = z.reshape(img_x, img_y).T #カーネル分布(合計=1)
    img_gray = np.uint8((min_max(data))*255) #標準化してグレースケールに変換
    return img_gray

In [ ]:
######################
#パノラマ画像＆カーネル密度生成
######################

for pointid, urls in enumerate(URL_lists): #i=pointid
    images = getAPIImgs(pointid, urls) #一地点の複数API画像をリストで取得する
    culcPosition(urls) #パノラマ画像の中心と右端の位置を求めリストに格納する
    R_list = getRList(pointid, urls) #各URLに対応する平面画像→パノラマ画像に変換するための回転行列を求める
    pano_img = genPanoImg(images, R_list) #パノラマ画像を生成する
    
    gaze_list_pano = getGazeListPano(pointid, Gaze_lists[pointid]) #視線データのパノラマ画像上の極座標を求める
    img_gray = Karnel(pano_img, gaze_list_pano, pointid) #カーネル密度を生成する
    
    cv2.imwrite("output/パノラマ画像/{}.png".format(str(pointid)), pano_img) #パノラマ画像出力
    cv2.imwrite("output/カーネル密度/{}.png".format(str(pointid)), img_gray) #カーネル密度出力